# SUP ML 3 - PREDICT

# Librerias

In [2]:
pip install shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 533.5/533.5 kB 7.3 MB/s eta 0:00:00


In [3]:
pip install dill

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 2.5 MB/s eta 0:00:00


In [44]:
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action = 'ignore', category = FutureWarning)

# imports best practice pandas
import os

import numpy as np
import pandas as pd
import missingno as msno
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import joblib
import dill

#--------------------------------------------------------
# imports best practice sklearn
import sklearn
from sklearn.feature_selection import VarianceThreshold
from sklearn import set_config

from sklearn.tree import DecisionTreeClassifier

# preprocessing
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from scipy import stats
from sklearn.preprocessing import LabelEncoder


from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer

# evaluacion
from sklearn.metrics import mean_absolute_error, r2_score, roc_curve, confusion_matrix, ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import metrics

#interpretabilidad
import shap

# modelos
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier

# pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
set_config(transform_output = "pandas")

# Carga modelo

* Cargar el modelo
* Obtener la lista de model features

In [5]:
#from google.colab import drive
#drive.mount('/content/drive')

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
CWD = os.getcwd()

DATA_PATH = os.path.join(CWD, r"/content/drive/MyDrive/NUCLIO/Machine Learning/Segundo entregable - ML/EJERCICIO-ML-Sup")

model = pickle.load(open(os.path.join(DATA_PATH, '/content/drive/MyDrive/NUCLIO/Machine Learning/Segundo entregable - ML/EJERCICIO-ML-Sup/classifier.pkl'), 'rb'))

In [8]:
# Podemos acceder al modelo booster_ del modelo de Gradient Boosting de LightGBM
# para recuperar el nombre de las características son consideradas por el modelo durante la predicción.

# model.feature_names_in_ para otros modelos

boost = model.booster_
features_model = boost.feature_name()
features_model

['rev',
 'mou',
 'totmrc',
 'da',
 'ovrmou',
 'ovrrev',
 'vceovr',
 'datovr',
 'roam',
 'change_mou',
 'change_rev',
 'drop_vce',
 'drop_dat',
 'blck_vce',
 'blck_dat',
 'unan_vce',
 'unan_dat',
 'plcd_vce',
 'plcd_dat',
 'recv_vce',
 'recv_sms',
 'comp_vce',
 'comp_dat',
 'custcare',
 'ccrndmou',
 'cc_mou',
 'inonemin',
 'threeway',
 'mou_cvce',
 'mou_cdat',
 'mou_rvce',
 'owylis_vce',
 'mouowylisv',
 'iwylis_vce',
 'mouiwylisv',
 'peak_vce',
 'peak_dat',
 'mou_peav',
 'mou_pead',
 'opk_vce',
 'opk_dat',
 'mou_opkv',
 'mou_opkd',
 'drop_blk',
 'attempt',
 'complete',
 'callfwdv',
 'callwait',
 'months',
 'uniqsubs',
 'actvsubs',
 'totcalls',
 'totmou',
 'totrev',
 'adjrev',
 'adjmou',
 'adjqty',
 'avgrev',
 'avgmou',
 'avgqty',
 'avg3mou',
 'avg3qty',
 'avg3rev',
 'avg6mou',
 'avg6qty',
 'avg6rev',
 'hnd_price',
 'phones',
 'models',
 'lor',
 'adults',
 'income',
 'numbcars',
 'eqpdays',
 'asl_flag_BOOL',
 'new_cell_N',
 'new_cell_U',
 'new_cell_Y',
 'crclscod_A',
 'crclscod_AA',
 'cr

# Carga PREDICT dataset

In [9]:
CWD = os.getcwd()

DATA_PATH = os.path.join(CWD, r"/content/drive/MyDrive/NUCLIO/Machine Learning/Segundo entregable - ML/EJERCICIO-ML-Sup")
df_desc = pd.read_csv("/content/drive/MyDrive/NUCLIO/Machine Learning/Segundo entregable - ML/EJERCICIO-ML-Sup/data/variable_dictionary.csv")
X_pred = pd.read_csv(os.path.join(DATA_PATH, "/content/drive/MyDrive/NUCLIO/Machine Learning/Segundo entregable - ML/EJERCICIO-ML-Sup/data/telecom_churn_PREDICT.csv"))

In [10]:
X_pred.shape

(10000, 99)

In [11]:
#Facilitar la compreensión de las variables y su tipo original
tipos = X_pred.dtypes
df_desc.drop(df_desc[df_desc["Variable"]=="churn"].index, axis=0, inplace=True)
df_desc.insert(loc=df_desc.shape[1], column="Tipo", value=tipos.values)

In [12]:

#Funciones Utiles

def listarNulos(dataset):
    #df_stat = dataset.isnull().sum()[(dataset.isnull().sum()>0)].sort_values(ascending=False).to_frame()
    #df_stat = dataset.isnull().sum().sort_values(ascending=False).to_frame()
    df_stat = dataset.isnull().sum().to_frame()
    df_stat.rename(columns={0: 'Nulos'}, inplace = True)
    percent = (df_stat["Nulos"]/dataset.shape[0]).mul(100)
    df_stat["Porcentaje"] = percent.round(2)
    tipos = dataset.dtypes
    df_stat["Tipo"] = df_stat.index.map(tipos)
    return df_stat[df_stat["Nulos"]>0].sort_values(by="Nulos", ascending=False)

def obtener_lista_variables(dataset, target):
    lista_numericas=[]
    lista_boolean=[]
    lista_categoricas=[]
    for i in dataset:
        if(i in dataset.select_dtypes("number")) and (i not in target):
              if (len(dataset[i].dropna().unique())!= 2):
                   lista_numericas.append(i)
              else:
                   lista_boolean.append(i)
        elif (i in dataset.select_dtypes("object")) and (i not in target):
              lista_categoricas.append(i)

    return lista_numericas, lista_boolean, lista_categoricas

def var_desc (lista):
    lista_retorno=list()
    retorno = df_desc[df_desc['Variable'].isin(lista)]
    return retorno

def plot_categorics(dataset, ord_unique):

    n_cols = 5
    n_rows = math.ceil(len(dataset)/n_cols)

    fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(20, 10))


    if (ord_unique):
         lista_categoricas_ord_unique = df_churn[dataset].describe().T.sort_values(by="unique").index.to_list()
         data_show = dataset[lista_categoricas_ord_unique]
    else:
         data_show = dataset


    for i, col in enumerate(data_show):
        row_plot = i // n_rows
        col_plot = i % n_cols
        title_plot = df_desc[df_desc["Variable"]==col][" Description"].values[0]
        sns.countplot(x=col, data=data_show, ax=axes[row_plot,col_plot]).set_title(title_plot)


    # Ajustar o layout
    plt.tight_layout()
    plt.show()

    return

def reportFeature(dataset, nan_only=False, dataset_merge=df_desc):
    #df_stat = dataset.isnull().sum()[(dataset.isnull().sum()>0)].sort_values(ascending=False).to_frame()
    #df_stat = dataset.isnull().sum().sort_values(ascending=False).to_frame()
    df_stat = dataset.isnull().sum().to_frame()
    df_stat.rename(columns={0: 'Nulos'}, inplace = True)
    percent = (df_stat["Nulos"]/dataset.shape[0]).mul(100)
    df_stat["Porcentaje"] = percent.round(2)
    tipos = dataset.dtypes
    df_stat["Tipo"] = df_stat.index.map(tipos)
    for i, val in enumerate(df_stat.index):
        if(val in dataset_merge["Variable"].values):
            df_stat.loc[val,"Description"] = dataset_merge[dataset_merge["Variable"]==val][" Description"].values[0]

    for i, val in enumerate(df_stat.index):
        df_stat.loc[val,"N_Unique"] = dataset[val].nunique()
        df_stat.loc[val,"Unique(10)"] = str(dataset[val].unique()[:10])

    if(nan_only):
         return df_stat[df_stat["Nulos"]>0].sort_values(by="Nulos", ascending=False)
    else:
         return df_stat.sort_values(by="Nulos", ascending=False)


def logCleaningOperation(dataset_log, dataset_main, list_variable, cat_operation, operation):

    for i in list_variable:
        tipo =  dataset_main.loc[i,"Tipo"]
        dataset_log.loc[dataset_log.shape[0]] = [i, tipo, cat_operation, operation]

    return dataset_log



In [13]:
#df_log

In [14]:
reportFeature(X_pred).sort_index()


,Nulos,Porcentaje,Tipo,Description,N_Unique,Unique(10)
Customer_ID,0,0.00,int64,Customer ID (Primary key),10000.0,[1090001 1090002 1090003 1090004 1090005 10900...
HHstatin,4891,48.91,object,Premier household status indicator,6.0,['C' nan 'I' 'A' 'G' 'H' 'B']
actvsubs,0,0.00,int64,Number of active subscribers in household,9.0,[1 2 3 5 4 0 7 6 9]
adjmou,0,0.00,float64,Billing adjusted total minutes of use over th...,6779.0,[ 958. 3998. 1772. 18478. 2601. 1400. 96...
adjqty,0,0.00,int64,Billing adjusted total number of calls over t...,4162.0,[ 393 1634 734 9018 791 732 3568 701 1501 ...
...,...,...,...,...,...,...
truck,281,2.81,float64,Truck indicator,2.0,[ 0. 1. nan]
unan_dat,0,0.00,float64,Mean number of unanswered data calls,27.0,[ 0. 1.33333333 0.33333333 0.666666...
unan_vce,0,0.00,float64,Mean number of unanswered voice calls,637.0,[ 13.33333333 10. 20.66666667 329.33...
uniqsubs,0,0.00,int64,Number of unique subscribers in the household,11.0,[ 1 2 3 4 6 5 7 8 10 12]


In [15]:
#df_log["Operacion"].unique()

# ML Preprocessing

Se deben realizar las mismas transformaciones que se aplicaron en el preprocessing del train:
  * Eliminar mismas variables
  * Imputar mismos valores a nulos
  * Aplicar mismo encoding de categoricos
  * ...todo lo necesario para replicar las variables con las que el modelo se entrenó

NO SE DEBEN repetir:
  * Analisis visual de los datos
  * Analisis de correlaciones
  * Analisis de varianza
  * Calculo de metricas que varien con la distribución

In [16]:
X_pred.head()

,rev,mou,totmrc,da,ovrmou,ovrrev,vceovr,datovr,roam,change_mou,...,forgntvl,ethnic,kid0_2,kid3_5,kid6_10,kid11_15,kid16_17,creditcd,eqpdays,Customer_ID
0,30.8350,136.75,29.9900,0.2475,1.25,0.500,0.5,0.000,0.0975,48.25,...,1.0,U,U,U,U,U,U,Y,216.0,1090001
1,35.8475,352.75,24.2700,0.4950,23.25,9.285,8.7,0.585,1.8000,-352.75,...,0.0,N,U,U,U,U,U,N,101.0,1090002
2,30.3275,241.50,39.9900,0.0000,0.00,0.000,0.0,0.000,0.0000,-86.50,...,0.0,N,U,U,U,U,U,Y,262.0,1090003
3,154.6925,2297.00,149.9900,4.7025,0.00,0.000,0.0,0.000,0.0000,119.00,...,0.0,S,U,U,U,U,U,Y,127.0,1090004
4,156.0050,542.25,48.9475,0.7425,70.75,28.295,28.1,0.195,0.0000,195.75,...,0.0,N,U,U,U,U,U,Y,37.0,1090005


In [21]:
X_pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 99 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   rev               9777 non-null   float64
 1   mou               9777 non-null   float64
 2   totmrc            9777 non-null   float64
 3   da                9777 non-null   float64
 4   ovrmou            9777 non-null   float64
 5   ovrrev            9777 non-null   float64
 6   vceovr            9777 non-null   float64
 7   datovr            9777 non-null   float64
 8   roam              9777 non-null   float64
 9   change_mou        9704 non-null   float64
 10  change_rev        9704 non-null   float64
 11  drop_vce          10000 non-null  float64
 12  drop_dat          10000 non-null  float64
 13  blck_vce          10000 non-null  float64
 14  blck_dat          10000 non-null  float64
 15  unan_vce          10000 non-null  float64
 16  unan_dat          10000 non-null  float64

- Eliminar filas duplicadas (si tenemos ID, solo puede haber 1 registro por ID)


In [22]:
# Verigicación de duplicados en la variable Customer_ID:

len(X_pred['Customer_ID'].unique()) - len(X_pred)


0

- Eliminar filas/columnas vacías y columnas irrelevantes

In [24]:
# Eliminar filas/columnas vacías

print((X_pred.isna().sum(axis=1)==X_pred.shape[1]).sum(),'all-empty rows were deleted')
X_pred.dropna(axis=0, how='all',inplace=True)
print(X_pred.shape)

# No hay columnas vacías

0 all-empty rows were deleted
(10000, 99)


In [25]:
#Eliminar filas/columnas vacías y columnas irrelevantes

#La variable "infobase" solo aporta un valor "M" además de nulos (21,5%)
df_report = reportFeature(X_pred).filter(like="infobase", axis=0)
df_report

,Nulos,Porcentaje,Tipo,Description,N_Unique,Unique(10)
infobase,2676,26.76,object,InfoBase match,2.0,['M' nan 'N']


In [26]:
#Eliminacion de la variable "infobase"
X_pred.drop(["infobase"], axis=1, inplace=True)


In [28]:
X_pred.shape

(10000, 98)

- **Variables numéricas con valor negativo**

In [29]:
# Reemplazo de valores negativos por la moda de 'eqpdays'

X_pred_eqp_days_less_than_zero = X_pred['eqpdays']<0
X_pred_eqp_days_not_na = X_pred['eqpdays'].notna()
mode_eqpdays = X_pred['eqpdays'].mode()[0]

Tratamiento de nulos

- **Variables categóricas**

In [30]:
#Imputar nulos con la etiqueta "U":
lista_var = ['prizm_social_one', 'kid0_2', 'kid3_5', 'kid16_17', 'kid11_15', 'kid6_10', 'marital', 'ethnic', 'dualband']

for i in lista_var:
    X_pred[i].fillna("U",inplace=True)

#Verificación
(
    reportFeature(X_pred, nan_only=True)
    [reportFeature(X_pred, nan_only=True)["Unique(10)"].str.contains("U")]
)

,Nulos,Porcentaje,Tipo,Description,N_Unique,Unique(10)
hnd_webcap,670,6.7,object,Handset web capability,3.0,['WCMB' 'UNKW' nan 'WC']


In [36]:
# Imputar nulos la misma etiqueta 'U' para nulos que figuran como nan",


# Columnas con nulos a imputar como 'U'
lista_var = ['prizm_social_one', 'kid0_2', 'kid3_5', 'kid16_17', 'kid11_15', 'kid6_10', 'marital', 'ethnic', 'dualband','area']

# Imputar nulos con la etiqueta 'U'
for col in lista_var:
    X_pred[col].fillna("U", inplace=True)

In [37]:
# Imputar nulos de hnd_webcap con la etiqueta 'UNKW'

X_pred['hnd_webcap'].fillna("UNKW", inplace=True)

In [38]:
# Imputación de nulos con etiqueta 'U' para el resto de
# Lista de variables a procesar
lista_var = ['dwllsize', 'HHstatin', 'ownrent', 'dwlltype', 'creditcd', 'refurb_new']

# Imputar nulos con la etiqueta 'U' para categorías con 1 caracter
for col in lista_var:
    X_pred[col].fillna("U", inplace=True)

- **Variables numéricas**

In [39]:
#Las variables numericas tienen distribucion asimétrica a la derecha, por lo que aparenta ser recomendable la utilización de la mediana
#para imputación de nulos

#en distribuciones asimétricas hacia la derecha, donde la media se ve afectada por valores atípicos o extremadamente altos,
#la mediana es una medida más robusta y a menudo preferible como medida central

#Registro de Log
category_operation = "Data Cleaning"
operation = "Imputación de nulos utilizando la mediana en variables numéricas con distribución asimétrica a la derecha"

# Descripción de las estadísticas para obtener la distribución de las columnas
df_estats = X_pred.describe().T

# Obtener las columnas numéricas
numeric_columns = df_estats.index

# Iterar sobre las columnas y rellenar los valores nulos con la mediana de cada columna
for column in numeric_columns:
    median_col = X_pred[column].median()
    X_pred[column].fillna(median_col, inplace=True)


In [40]:
#Las variables "forgntvl", "rv", "truck" son binárias entre 0 y 1.
#Por lo tanto, pueden ser convertidas a "int" y los nulos imputados con outlier para ese valor (-1)

#"Convertir FLOAT en INT - variable binaria [1,0]"
lista_var = ["forgntvl", "rv", "truck"]
for i in lista_var:
    X_pred[i] = X_pred[i].astype("int8")
    X_pred[i] = X_pred[i].astype("int8")
    X_pred[i] = X_pred[i].astype("int8")

Encoding categóricos

In [41]:
#Las variables con valores entre "Y" y "U" parecen no ser representativa porque, efectivamente, solo contienen "Y" además de "U" - nulo. És decir, o el valor es "Y", o no es conocido.
# "Eliminar variables con valores entre 'Y' y 'U' "

lista_drop = ['kid16_17', 'kid11_15', 'kid6_10', 'kid3_5', 'kid0_2']
X_pred.drop(columns=lista_drop,inplace=True)

In [42]:
#Categorización de las áreas en la variable "area" por Regioneas Administrativas* - disminución de cardinalidad
dict_area = {"NEW ENGLAND AREA": "NORTHEAST" ,
             "PHILADELPHIA AREA":"MID-ATLANTIC",
             "NEW YORK CITY AREA": "MID-ATLANTIC",
              "TENNESSEE AREA": "SOUTH",
              "ATLANTIC SOUTH AREA": "SOUTH",
              "SOUTH FLORIDA AREA": "SOUTH",
              "DC/MARYLAND/VIRGINIA AREA": "SOUTH",
              "NORTH FLORIDA AREA": "SOUTH",
               "CHICAGO AREA": "MIDWEST",
                "GREAT LAKES AREA": "MIDWEST",
                "MIDWEST AREA": "MIDWEST",
                "OHIO AREA": "MIDWEST",
                "NORTHWEST/ROCKY MOUNTAIN AREA": "WEST",
                "CALIFORNIA NORTH AREA": "WEST",
                "LOS ANGELES AREA": "WEST",
                "DALLAS AREA": "SOUTHWEST",
                "CENTRAL/SOUTH TEXAS AREA": "SOUTHWEST",
                "HOUSTON AREA": "SOUTHWEST",
                "SOUTHWEST AREA": "SOUTHWEST",}


X_pred["area_ENCOD"] = X_pred["area"].replace(dict_area)
X_pred["area_ENCOD"].unique()

# Eliminación de la variable area
X_pred.drop(columns=["area"], inplace=True)

In [45]:
#Variable "asl_flag" es binaria entre 'Y' y 'N'.

#"Aplicar identificador binario ['Y', 'N'] - Variable _BOOL (INT) - LabelEncoder()."

le = LabelEncoder()
X_pred["asl_flag_BOOL"] = le.fit_transform(X_pred["asl_flag"])
X_pred.drop(columns=["asl_flag"], inplace=True)

In [46]:
df_report=\
(
reportFeature(X_pred)
[reportFeature(X_pred)["Tipo"]=="object"]
.sort_index()
)
df_report

,Nulos,Porcentaje,Tipo,Description,N_Unique,Unique(10)
HHstatin,0,0.0,object,Premier household status indicator,7.0,['C' 'U' 'I' 'A' 'G' 'H' 'B']
area_ENCOD,0,0.0,object,NaN,6.0,['SOUTH' 'WEST' 'SOUTHWEST' 'MIDWEST' 'MID-ATL...
crclscod,0,0.0,object,Credit class code,45.0,['AA' 'EA' 'A' 'CA' 'DA' 'ZA' 'BA' 'B' 'GA' 'E']
creditcd,0,0.0,object,Credit card indicator,3.0,['Y' 'N' 'U']
dualband,0,0.0,object,Dualband,4.0,['Y' 'T' 'N' 'U']
dwllsize,0,0.0,object,Dwelling size,16.0,['U' 'A' 'B' 'F' 'C' 'N' 'M' 'J' 'G' 'O']
dwlltype,0,0.0,object,Dwelling Unit type,3.0,['M' 'U' 'S']
ethnic,0,0.0,object,Ethnicity roll-up code,17.0,['U' 'N' 'S' 'H' 'O' 'B' 'Z' 'D' 'I' 'G']
hnd_webcap,0,0.0,object,Handset web capability,3.0,['WCMB' 'UNKW' 'WC']
marital,0,0.0,object,Marital Status,5.0,['M' 'U' 'S' 'B' 'A']


In [47]:
X_pred2 = X_pred.copy()
#X_pred = X_pred2.copy()

In [48]:
df_report=\
(
reportFeature(X_pred)
[reportFeature(X_pred)["Tipo"]=="object"]
.sort_index()
)

#Aplicar valor -1 para cada valor "U" (Outlier - etiqueta para nulo/"Unknown")
lista_var = df_report.index
for i in lista_var:
    col=i+"_BOOL"
    if ("U" in X_pred[i].values):
        X_pred[col] = X_pred[i].replace("U","-1")
    elif ("UNKNOWN" in X_pred[i].values):
        X_pred[col] = X_pred[i].replace("UNKNOWN","-1")
    elif ("UNKW" in X_pred[i].values):
        X_pred[col] = X_pred[i].replace("UNKW","-1")

#corrección
X_pred["area_ENCOD_BOOL"] = X_pred["area_ENCOD"]
X_pred["refurb_new_BOOL"] = X_pred["refurb_new"]

X_pred.drop(columns=lista_var, inplace=True)

In [49]:
df_report=(
reportFeature(X_pred)
[reportFeature(X_pred)["Tipo"]=="object"]
.sort_index()
)
df_report

,Nulos,Porcentaje,Tipo,Description,N_Unique,Unique(10)
HHstatin_BOOL,0,0.0,object,NaN,7.0,['C' '-1' 'I' 'A' 'G' 'H' 'B']
area_ENCOD_BOOL,0,0.0,object,NaN,6.0,['SOUTH' 'WEST' 'SOUTHWEST' 'MIDWEST' 'MID-ATL...
crclscod_BOOL,0,0.0,object,NaN,45.0,['AA' 'EA' 'A' 'CA' 'DA' 'ZA' 'BA' 'B' 'GA' 'E']
creditcd_BOOL,0,0.0,object,NaN,3.0,['Y' 'N' '-1']
dualband_BOOL,0,0.0,object,NaN,4.0,['Y' 'T' 'N' '-1']
dwllsize_BOOL,0,0.0,object,NaN,16.0,['-1' 'A' 'B' 'F' 'C' 'N' 'M' 'J' 'G' 'O']
dwlltype_BOOL,0,0.0,object,NaN,3.0,['M' '-1' 'S']
ethnic_BOOL,0,0.0,object,NaN,17.0,['-1' 'N' 'S' 'H' 'O' 'B' 'Z' 'D' 'I' 'G']
hnd_webcap_BOOL,0,0.0,object,NaN,3.0,['WCMB' '-1' 'WC']
marital_BOOL,0,0.0,object,NaN,5.0,['M' '-1' 'S' 'B' 'A']


In [50]:
#Aplicar OHE para variables no binarias y no ordinales
df_report=(
reportFeature(X_pred)
[reportFeature(X_pred)["Tipo"]=="object"]
.sort_index()
)

lista_var = df_report.index
lista_ohe = list()
for i in lista_var:
    one_hot_df = pd.get_dummies(X_pred[i], prefix=i)
    lista_ohe += one_hot_df.columns.to_list()
    X_pred = pd.concat([X_pred, one_hot_df], axis=1)


In [51]:
X_pred.head().sort_index(axis=1)

,Customer_ID,HHstatin_BOOL,HHstatin_BOOL_-1,HHstatin_BOOL_A,HHstatin_BOOL_B,HHstatin_BOOL_C,HHstatin_BOOL_G,HHstatin_BOOL_H,HHstatin_BOOL_I,actvsubs,...,threeway,totcalls,totmou,totmrc,totrev,truck,unan_dat,unan_vce,uniqsubs,vceovr
0,1090001,C,0,0,0,1,0,0,0,1,...,0.333333,393,958.0,29.9900,248.46,0,0.0,13.333333,1,0.5
1,1090002,-1,1,0,0,0,0,0,0,1,...,1.333333,1641,4011.0,24.2700,391.87,0,0.0,10.000000,1,8.7
2,1090003,C,0,0,0,1,0,0,0,2,...,1.666667,780,1889.0,39.9900,312.91,1,0.0,20.666667,2,0.0
3,1090004,-1,1,0,0,0,0,0,0,1,...,0.000000,9018,18478.0,149.9900,2217.05,0,0.0,329.333333,1,0.0
4,1090005,C,0,0,0,1,0,0,0,1,...,0.333333,791,2601.0,48.9475,879.51,0,0.0,46.333333,1,28.1


In [52]:
#Eliminación de las columnas de M1 por alta correlación
#variables eliminadas en M1 :
list_corr_m1_to_drop = ['ovrmou', 'ovrrev', 'plcd_vce', 'comp_dat', 'cc_mou', 'inonemin',
       'mou_opkd', 'attempt', 'comp_vce', 'totrev', 'totmou', 'totcalls',
       'avg3mou', 'avg6mou', 'avg6qty']

X_pred.drop(columns=list_corr_m1_to_drop, inplace=True)


In [53]:
#Eliminación de las columnas de M1 por baja varianza
list_cols_lowvar_M1_to_Drop = ['refurb_new_BOOL_-1', 'HHstatin_BOOL_H', 'ethnic_BOOL_D', 'ethnic_BOOL_M', 'ethnic_BOOL_P', 'ethnic_BOOL_X', 'area_ENCOD_BOOL_-1', 'dualband_BOOL_-1']

for i in (list_cols_lowvar_M1_to_Drop):
    if(i in X_pred.columns):
        X_pred.drop(columns=i,inplace=True)

# Check model features

* Comprobar que tenemos en el dataset preprocesado todas las model features, de lo contrario no podremos hacer predict.
* Ordenar las variables en mismo orden que las model features

In [ ]:
# X_pred= pd.read_pickle("data/X_pred")


In [54]:
features_test = list(X_pred.columns)

print('Columnas en dataset:',len(features_test))
print('Variables en modelo:',len(features_model))
print('¿Match?:', features_model == features_test)

Columnas en dataset: 194
Variables en modelo: 146
¿Match?: False


In [55]:
missing_features = [i for i in features_model if i not in features_test]
print('Variables que faltan el el dataset:\n', missing_features)

Variables que faltan el el dataset:
 ['ovrmou', 'ovrrev', 'plcd_vce', 'comp_vce', 'comp_dat', 'cc_mou', 'inonemin', 'mou_opkd', 'attempt', 'totcalls', 'totmou', 'totrev', 'avg3mou', 'avg6mou', 'avg6qty', 'new_cell_N', 'new_cell_U', 'new_cell_Y', 'crclscod_A', 'crclscod_AA', 'crclscod_B', 'crclscod_BA', 'crclscod_C', 'crclscod_CA', 'crclscod_DA', 'crclscod_EA', 'crclscod_ZA', 'asl_flag_N', 'asl_flag_Y', 'prizm_social_one_C', 'prizm_social_one_R', 'prizm_social_one_S', 'prizm_social_one_T', 'prizm_social_one_U', 'dualband_N', 'dualband_T', 'dualband_Y', 'refurb_new_N', 'refurb_new_R', 'hnd_webcap_UNKW', 'hnd_webcap_WC', 'hnd_webcap_WCMB', 'ownrent_O', 'ownrent_R', 'ownrent_U', 'dwlltype_M', 'dwlltype_S', 'dwlltype_U', 'marital_A', 'marital_B', 'marital_M', 'marital_S', 'marital_U', 'HHstatin_A', 'HHstatin_B', 'HHstatin_C', 'HHstatin_G', 'HHstatin_I', 'HHstatin_U', 'dwllsize_A', 'dwllsize_B', 'dwllsize_C', 'dwllsize_J', 'dwllsize_O', 'dwllsize_U', 'ethnic_B', 'ethnic_F', 'ethnic_G', 'ethn

In [56]:
# agregamos columnas faltantes

for col in missing_features:
  X_pred[col]= 0

In [57]:
drop_features = [i for i in features_test if i not in features_model]
print('Variables que debes eliminar de tu dataset:\n', drop_features)

Variables que debes eliminar de tu dataset:
 ['truck', 'rv', 'forgntvl', 'Customer_ID', 'HHstatin_BOOL', 'crclscod_BOOL', 'creditcd_BOOL', 'dualband_BOOL', 'dwllsize_BOOL', 'dwlltype_BOOL', 'ethnic_BOOL', 'hnd_webcap_BOOL', 'marital_BOOL', 'new_cell_BOOL', 'ownrent_BOOL', 'prizm_social_one_BOOL', 'area_ENCOD_BOOL', 'refurb_new_BOOL', 'HHstatin_BOOL_-1', 'HHstatin_BOOL_A', 'HHstatin_BOOL_B', 'HHstatin_BOOL_C', 'HHstatin_BOOL_G', 'HHstatin_BOOL_I', 'area_ENCOD_BOOL_MID-ATLANTIC', 'area_ENCOD_BOOL_MIDWEST', 'area_ENCOD_BOOL_NORTHEAST', 'area_ENCOD_BOOL_SOUTH', 'area_ENCOD_BOOL_SOUTHWEST', 'area_ENCOD_BOOL_WEST', 'crclscod_BOOL_-1', 'crclscod_BOOL_A', 'crclscod_BOOL_A2', 'crclscod_BOOL_AA', 'crclscod_BOOL_B', 'crclscod_BOOL_B2', 'crclscod_BOOL_BA', 'crclscod_BOOL_C', 'crclscod_BOOL_C2', 'crclscod_BOOL_C5', 'crclscod_BOOL_CA', 'crclscod_BOOL_CC', 'crclscod_BOOL_CY', 'crclscod_BOOL_D', 'crclscod_BOOL_D4', 'crclscod_BOOL_D5', 'crclscod_BOOL_DA', 'crclscod_BOOL_E', 'crclscod_BOOL_E2', 'crclsco

In [58]:
X_pred.drop(columns = drop_features, inplace=True)

In [59]:
features_test = list(X_pred.columns)

print('Columnas en dataset:',len(features_test))
print('Variables en modelos:',len(features_model))
print('¿Match?:', features_model == features_test)

Columnas en dataset: 146
Variables en modelos: 146
¿Match?: False


In [60]:
# Reordena variables

X_pred = X_pred[features_model]

In [61]:
features_test = list(X_pred.columns)

print('Columnas en dataset:',len(features_test))
print('Variables en modelos:',len(features_model))
print('¿Match?:', features_model == features_test)

Columnas en dataset: 146
Variables en modelos: 146
¿Match?: True


# Rescaling

* Si se entrenó el modelo con un dataset estandarizado, estandarizar con mismo scaler.

In [ ]:
#No hace falta estandarizar para el modelo utilizado

# PREDICT

In [62]:
predictions = model.predict(X_pred)

In [63]:
# prediccion de churn en mes + 1

predictions

array([0, 1, 0, ..., 1, 1, 0])

In [64]:
predict_proba = model.predict_proba(X_pred)

In [65]:
# probabilidad de no churn mes + 1 (izquierda) y churn en mes + 1 (derecha)

predict_proba

array([[0.57494766, 0.42505234],
       [0.44893584, 0.55106416],
       [0.699424  , 0.300576  ],
       ...,
       [0.46695593, 0.53304407],
       [0.46935348, 0.53064652],
       [0.6024908 , 0.3975092 ]])

In [66]:
# probabilidad de churn en mes + 1

predict_proba[:,1]

array([0.42505234, 0.55106416, 0.300576  , ..., 0.53304407, 0.53064652,
       0.3975092 ])

In [67]:
X_pred.head()

,rev,mou,totmrc,da,ovrmou,ovrrev,vceovr,datovr,roam,change_mou,...,ethnic_Z,creditcd_N,creditcd_U,creditcd_Y,area_ENCOD_MID-ATLANTIC,area_ENCOD_MIDWEST,area_ENCOD_NORTHEAST,area_ENCOD_SOUTH,area_ENCOD_SOUTHWEST,area_ENCOD_WEST
0,30.8350,136.75,29.9900,0.2475,0,0,0.5,0.000,0.0975,48.25,...,0,0,0,0,0,0,0,0,0,0
1,35.8475,352.75,24.2700,0.4950,0,0,8.7,0.585,1.8000,-352.75,...,0,0,0,0,0,0,0,0,0,0
2,30.3275,241.50,39.9900,0.0000,0,0,0.0,0.000,0.0000,-86.50,...,0,0,0,0,0,0,0,0,0,0
3,154.6925,2297.00,149.9900,4.7025,0,0,0.0,0.000,0.0000,119.00,...,0,0,0,0,0,0,0,0,0,0
4,156.0050,542.25,48.9475,0.7425,0,0,28.1,0.195,0.0000,195.75,...,0,0,0,0,0,0,0,0,0,0


In [68]:
X_pred.index

RangeIndex(start=0, stop=10000, step=1)

In [69]:
df_churn_final_predict = pd.DataFrame()

In [72]:
df_churn_final_predict['Customer_ID'] = X_pred.index


In [76]:
df_churn_final_predict['Customer_ID'] = X_pred.index

df_churn_final_predict['PREDICT'] = predictions

df_churn_final_predict['PREDICT_PROBA'] = predict_proba[:,1]

In [77]:
df_churn_final_predict

,Customer_ID,PREDICT,PREDICT_PROBA
0,0,0,0.425052
1,1,1,0.551064
2,2,0,0.300576
3,3,1,0.571426
4,4,1,0.513436
...,...,...,...
9995,9995,1,0.561350
9996,9996,1,0.538288
9997,9997,1,0.533044
9998,9998,1,0.530647


* predict() y predict_proba()

In [ ]:
#No hace falta estandarizar para el modelo utilizado

# Guarda predicciones

* Guardar las predicciones en data path. Cada fila debe estar etiquetada con el ID.

In [78]:
CWD = os.getcwd()

DATA_PATH = os.path.join(CWD, r"/content/drive/MyDrive/NUCLIO/Machine Learning/Segundo entregable - ML/EJERCICIO-ML-Sup")

df_churn_final_predict.to_csv(os.path.join(DATA_PATH, 'df_churn_final_predict.csv'))
